Credit: Đoàn Vĩnh Nhân - Toán Tin K68 - 20237376

### Phương pháp khử Gauss
+ Input: ma trận $A_{m\times n}, B_{m\times q}$
+ Output: ma trận $X_{n\times (q+k)}$, với $k$ là số hàng bằng 0 sau khi khử $AB$. Các phần tử trên cột q+1, q+2,...q+k đặc trưng cho sự phụ thuộc của các phần tử còn lại của $X$ vào các phần tử tự do.

Ví dụ với nghiệm:
$$\begin{array}{ccccc}
&&&&X_3&X_4\\
&1&2&3&4&5\\
&2&3&4&5&6\\
&3&4&5&6&7
\end{array}$$
Khi đó $X=\begin{bmatrix}X_1&X_2&X_3&X_4&X5\end{bmatrix}^\top$
\begin{array}{ll}
X_1&=\begin{pmatrix}1&2&3\end{pmatrix} + 4X_4 + 5X_5\\
X_2&=\begin{pmatrix}2&3&4\end{pmatrix} + 5X_4 + 6X_5\\
X_3&=\begin{pmatrix}3&4&5\end{pmatrix} + 6X_4 + 7X_5\\
X_4&=\begin{pmatrix}t_{41}&t_{42}&t_{43}\end{pmatrix}, \quad t \in \mathbb{R}\\
X_5&=\begin{pmatrix}t_{51}&t_{52}&t_{53}\end{pmatrix}, \quad t \in \mathbb{R}\\
\end{array}
Thuật toán
1. Khử ma trận $AB$
   + Bước 1: Tạo ma trận mở rộng $A|B$, tạo 1 danh sách index: lưu vị trí cột của các phần tử khác 0 đầu tiên trong từng hàng. Hoặc nếu là hàng đó bằng 0, lưu $n$
   + Bước 2: Sắp xếp lại index theo thứ tự không giảm, sắp xếp các hàng của ma trận mở rộng AB theo index => Để có ma trận bậc thang
   + Step 3: Khử ma trận, theo từng hàng => nếu chưa đến hàng cuối quay lại Step 2, nếu xong chuyển đến B2
2. Kiểm tra index của AB, nếu tồn tại phần tử chỉ đến cột của B trong AB => Kết luận vô nghiệm và dừng (trường hợp $rank(A)<rank(AB)$, hàng A bằng 0 nhưng hàng B có phần tử khác 0)
3. Chuyển các cột có phần tử trên đường chéo chính bằng $0$ của $A$ ra sau $B$
   + Bước 1: Dùng danh sách index, tìm kiếm những cột có phần tử trên đường chéo chính bằng 0
   + Bước 2: Chuyển các cột đó ra sau ma trận B
4. Giải tìm $X$ với $A$ mới và $B$ mới (Ma trận A giờ đây là ma trận tam giác trên)
   + Bước 1: Tạo ma trận không $X_{n\times (q+k)}$
   + Bước 2: Dùng vòng lặp col và row, tính toán từng phần tử của ma trận $X$

Ví dụ:
     $\begin{bmatrix}4&0&0\\0&2&1\\0&0&1\\5&2&1\\0&0&0\\0&3&2\end{bmatrix}$, index $= [0 ,1 ,2, 0, 3,1] $
$\Rightarrow$ Sắp xếp lại:
     $\begin{bmatrix}4&0&0\\5&2&1\\0&2&1\\0&3&2\\0&0&1\\0&0&0\end{bmatrix}$
          index $=[0 ,0 ,1, 1, 2, 3]$

In [3]:
from sympy import Matrix, zeros, N
import numpy as np

# Hàm này được thêm vào để loại bỏ các giá trị nhỏ hơn 10⁻¹⁰, đảm bảo chúng được coi là 0.
def chop_small_values(mat):
    if isinstance(mat, Matrix):
        return mat.applyfunc(lambda x: 0 if abs(x) < 1e-10 else x)
    else:
        return 0 if abs(mat) < 1e-10 else mat
#Danh sách các vị trí khác 0 đầu tiên của từng hàng, dùng để sắp xếp AB theo cấu trúc bậc thang
def indexpivot(AB):
    rowAB, colAB = AB.shape
    index = []
    for row in range(rowAB):
        for col in range(colAB):
            if abs(AB[row, col]) > 1e-10: 
                index.append(col)
                break
            if col == colAB - 1:
                index.append(col + 1)
    return index
    
#Vị trí các cột có phần tử ở đường chéo chính bằng 0
def missingindex(A):
    colA = A.shape[1]
    index = indexpivot(A)
    return list(set(range(colA)) - set(index))
    
#Khử gauss
def gauss(A, B):
    A, B = Matrix(N(A)), Matrix(N(B))
    AB = chop_small_values(A.row_join(B))
    rowAB, colAB = AB.shape
    
    #Sắp xếp lại AB theo cấu trúc bậc thang trong trường hợp input không phải là bậc thang    
    index = indexpivot(AB)
    sorted_order = sorted(range(len(index)), key=lambda k: index[k])
    index = sorted(index)
    AB = chop_small_values(AB[list(sorted_order), :])
    display(AB)

    #Khử Gauss (Vừa khử, vừa sắp xếp lại AB theo cấu trúc bậc thang)
    for i in range(len(index)):
        if index[i] != colAB:
            for j in range(i + 1, len(index)):
                AB[j, :] = chop_small_values(Matrix(AB[j, :] - AB[i, :] / AB[i, index[i]] * AB[j, index[i]]))
            index = indexpivot(AB)
            sorted_order = sorted(range(len(index)), key=lambda k: index[k])
            index = sorted(index)
            AB = chop_small_values(AB[list(sorted_order), :])
            print("Bước khử")
            display(AB)
    print("Ma trận sau khi khử:")
    display(AB)
    return AB

#Giải AX=B
def gauss_solve(A, B):
    #Ma trận đã khử AB
    A, B = Matrix(N(A)), Matrix(N(B))
    AB = gauss(A, B)

    #Kiểm tra trường hợp vô nghiệm
    for i in indexpivot(AB):
        if i >= A.shape[1] and i < AB.shape[1]:
            print("Hệ vô nghiệm")
            return 

    #Ma trận A, B mới từ ma trận đã khử    
    A = AB[:, :A.shape[1]]
    B = AB[:, A.shape[1]:]
    colB = B.shape[1]

    #Chuyển những cột có phần tử trên đường chéo chính bằng 0 sang B
    missindex = missingindex(A)
    if missindex:
        print(f"index: {[x + 1 for x in indexpivot(A)]}")
        print(f"missing index: {[x + 1 for x in missindex]}")
        for i in missindex:
            move_col = -1 * A[:, i]
            B = B.row_join(move_col)
        A = A[:, [i for i in range(A.shape[1]) if i not in missindex]]
        print("Ma trận sau khi chuyển cột:")
        display(chop_small_values(A.row_join(B)))

    #Tính toán ma trận X    
    X = zeros(A.shape[1], B.shape[1])
    rowX, colX = X.shape
    index = indexpivot(A)
    
    for col in range(colX):
        for row in range(rowX - 1, -1, -1):
            sum_val = sum(A[row, k] * X[k, col] for k in range(index[row], rowX))
            X[row, col] = chop_small_values((B[row, col] - sum_val) / A[row, index[row]])
    
    print(f"Số phần tử tự do là {len(missindex)}: X{[x + 1 for x in missindex]}. Nghiệm:")
    return X

In [7]:
A = Matrix(np.loadtxt("G_input_A.txt"))
print("A =")
display(A)
print("B =")
B = Matrix(np.loadtxt("G_input_B.txt"))
display(B)
print("A|B=")
X = gauss_solve(A,B)
display(X)

A =


Matrix([
[ 130.75,    3.75,    5.25,    11.25,   18.25,     6.5,    12.0,     15.5,    2.75,     20.0],
[   -7.0,   156.0,     4.0,      8.0, 17.6667, -3.6667,    -3.0,      4.0,     9.0,  21.6667],
[    2.8,     4.2,    89.0,      5.4,    10.8,    12.6,    -7.8,      8.0,    14.8,     13.0],
[16.5714, 14.8571, 25.4286, 140.4286, 25.7143,  2.1429,  5.5714,  10.5714,  5.4286,   6.4286],
[ 6.1667,   -19.0,  9.8333,  12.1667,   114.0, -6.1667,    14.5,   3.6667,   -10.5, -14.1667],
[ -0.875,  11.125,   7.125,    6.375,  13.375,   81.75,    -6.0,    1.875,  -15.75,    -8.75],
[-7.5714, -1.4286,  5.2857,   9.8571,   -18.0,  7.1429, 96.8571, -10.1429, -8.8571, -12.8571],
[   17.2,    11.0,    15.6,      2.8,     2.2,    14.2,    15.6,    106.8,     5.2,      6.4],
[15.8333,  4.8333, -5.8333,      5.5, -9.6667,  7.6667,  5.6667,      9.5, 81.1667,     -3.0],
[ 0.9167,    -2.0, -2.5833,   2.9167,  3.9167, 10.6667,  2.1667,   9.5833, -3.5833,  52.0833]])

B =


Matrix([
[ 310.2275],
[-193.5166],
[  238.112],
[-165.0229],
[ 214.4215],
[ 108.6325],
[  28.8543],
[  291.732],
[-256.5717],
[ 197.1857]])

A|B=


...

Matrix([
[130.75,             3.75,             5.25,            11.25,            18.25,               6.5,              12.0,              15.5,              2.75,              20.0,          310.2275],
[     0, 156.200764818356,  4.2810707456979, 8.60229445506692, 18.6437554493308, -3.31870764818356, -2.35755258126195,  4.82982791586998,   9.1472275334608,  22.7374456978967, -176.907861950287],
[     0,                0, 88.7746612316845, 4.93220226947229, 9.91746083728104,   12.548331825493, -7.99480004406742,   7.5406852484301,  14.4998567808747,  11.9720163071499,   236.13433513716],
[     0,                0,                0, 136.856814867376,  18.962307357916, -1.81992787753161,  6.46218103472839,  6.09226601944848, 0.257577110245947, -1.48609117744435, -252.873024771185],
[     0,                0,                0,                0, 112.617836498482, -8.14862799679924,  13.9824048337341,  2.12975305716963, -11.1810292296961, -13.5503611720962,   173.58919802336],
[     0,   

Ma trận sau khi khử:


Matrix([
[130.75,             3.75,             5.25,            11.25,            18.25,               6.5,              12.0,              15.5,              2.75,              20.0,          310.2275],
[     0, 156.200764818356,  4.2810707456979, 8.60229445506692, 18.6437554493308, -3.31870764818356, -2.35755258126195,  4.82982791586998,   9.1472275334608,  22.7374456978967, -176.907861950287],
[     0,                0, 88.7746612316845, 4.93220226947229, 9.91746083728104,   12.548331825493, -7.99480004406742,   7.5406852484301,  14.4998567808747,  11.9720163071499,   236.13433513716],
[     0,                0,                0, 136.856814867376,  18.962307357916, -1.81992787753161,  6.46218103472839,  6.09226601944848, 0.257577110245947, -1.48609117744435, -252.873024771185],
[     0,                0,                0,                0, 112.617836498482, -8.14862799679924,  13.9824048337341,  2.12975305716963, -11.1810292296961, -13.5503611720962,   173.58919802336],
[     0,   

Số phần tử tự do là 0: X[]. Nghiệm:


Matrix([
[ 1.52000019502473],
[-1.54000018097686],
[ 2.50000016823524],
[-2.14000031802933],
[ 1.49999993415033],
[ 0.99999985997587],
[0.999999530603812],
[  2.0000001226982],
[-3.15000035540775],
[ 2.99999923469958]])